In [1]:
with open("zero_shot_29APR.out", "r") as f:
    raw_data = f.readlines()

In [2]:
raw_data

['Memory usage before starting the job:\n',
 '              total        used        free      shared  buff/cache   available\n',
 'Mem:          376Gi        37Gi       132Gi        49Gi       205Gi       286Gi\n',
 'Swap:         4.0Gi       4.0Gi        22Mi\n',
 'Mon Apr 29 10:28:32 2024       \n',
 '+---------------------------------------------------------------------------------------+\n',
 '| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |\n',
 '|-----------------------------------------+----------------------+----------------------+\n',
 '| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |\n',
 '| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |\n',
 '|                                         |                      |               MIG M. |\n',
 '|=========================================+======================+======================|\n',
 '|   0  NVIDIA A40     

In [3]:
raw_data[20:28]

['|        ID   ID                                                             Usage      |\n',
 '|=======================================================================================|\n',
 '|  No running processes found                                                           |\n',
 '+---------------------------------------------------------------------------------------+\n',
 'Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.\n',
 'Token is valid (permission: read).\n',
 'Your token has been saved to /home/cjcal/.cache/huggingface/token\n',
 'Login successful\n']

In [4]:
raw_data = raw_data[28:]

In [5]:
raw_data[-1]

'The person put down the bottle of medicine before picking it up and holding it. The person did not throw, take, or open the container.\n'

In [6]:
len(raw_data)

58905

In [7]:
raw_data[:30]

['Instruction: Predict the answer based on the question.\n',
 'Question: Which object was tidied up by the person?\n',
 'Choices: \n',
 '(A) The closet/cabinet.\n',
 '(B) The blanket.\n',
 '(C) The clothes.\n',
 '(D) The table.\n',
 'Answer: The answer is  (A).\n',
 '\n',
 '### Explanation:\n',
 'The person is tidying up the room, so it is most likely that they are putting things away in a cabinet. The other choices are not likely to be the object that is being put away.\n',
 'Instruction: Predict the answer based on the question.\n',
 'Question: Which object was tidied up by the person?\n',
 'Choices: \n',
 '(A) The blanket.\n',
 '(B) The table.\n',
 '(C) The clothes.\n',
 '(D) The closet/cabinet.\n',
 'Answer: The answer is  (B).\n',
 'Explanation: In the picture, we can see that the table is tidy, but the other objects are messy. Therefore, the correct answer must be the one that is the opposite of what is shown in the image. In this case, it is "the table".\n',
 '\n',
 '### Questio

In [8]:
def find_matching_original(parsed_set, original_data):
    parsed_question = normalize(parsed_set['Question'])
    parsed_choices = sorted([normalize(choice) for choice in parsed_set['Choices']])

    for original in original_data:
        original_question = normalize(original['question'])
        original_choices = sorted([normalize(choice['choice']) for choice in original['choices']])

        if parsed_question == original_question and parsed_choices == original_choices:
            return {
                'question_id': original['question_id'],
                'video_id': original['video_id'],
                'correct_answer': original['answer']
            }
    # Print debug information if no match is found
    print(f"No match found for: {parsed_question} with choices {parsed_choices}")
    return None

In [9]:
import re
def parse_data(lines):
    results = []
    current = {}
    choices = []
    
    for line in lines:
        line = line.strip()
        if line.startswith('Instruction:'):
            if current: 
                current['Choices'] = choices
                results.append(current)
                choices = []
            current = {'Instruction': line[len('Instruction:'):].strip()}
        elif line.startswith('Question:'):
            current['Question'] = line[len('Question:'):].strip()
        elif line.startswith('('): 
            choices.append(line)
        elif line.startswith('Answer:'):
            match = re.search(r'\(.\)', line)
            if match:
                current['Answer'] = match.group(0)
    
    if current:
        current['Choices'] = choices
        results.append(current)
    
    return results



In [132]:
def parse_data(lines):
    """
    Updated to grab the explanation
    """
    results = []
    current = {}
    choices = []
    in_explanation = False
    explanation = []

    for line in lines:
        line = line.strip()
        if line.startswith('Instruction:'):
            if current:  
                if explanation: 
                    current['Explanation'] = ' '.join(explanation).strip()
                    explanation = []
                current['Choices'] = choices
                results.append(current)
                choices = []
            current = {'Instruction': line[len('Instruction:'):].strip()}
        elif line.startswith('Question:'):
            current['Question'] = line[len('Question:'):].strip()
        elif line.startswith('('): 
            choices.append(line)
        elif line.startswith('Answer:'):
            match = re.search(r'\(.\)', line)
            if match:
                current['Answer'] = match.group(0)
        elif line.startswith('Explanation:'):
            in_explanation = True
            explanation.append(line[len('Explanation:'):].strip())  
        elif in_explanation:
            if line == '' or line.startswith('###'): 
                in_explanation = False
            else:
                explanation.append(line) 

    if current:  
        if explanation:
            current['Explanation'] = ' '.join(explanation).strip()
        current['Choices'] = choices
        results.append(current)

    return results

In [10]:
parse_data(raw_data[:10])

[{'Instruction': 'Predict the answer based on the question.',
  'Question': 'Which object was tidied up by the person?',
  'Answer': '(A)',
  'Choices': ['(A) The closet/cabinet.',
   '(B) The blanket.',
   '(C) The clothes.',
   '(D) The table.']}]

In [133]:
parsed_data = parse_data(raw_data)

In [15]:
len(parsed_data)

5789

In [16]:
parsed_data[0]

{'Instruction': 'Predict the answer based on the question.',
 'Question': 'Which object was tidied up by the person?',
 'Answer': '(A)',
 'Choices': ['(A) The closet/cabinet.',
  '(B) The blanket.',
  '(C) The clothes.',
  '(D) The table.']}

In [17]:
!ls data/star

clipvitl14.pth	STAR_test.json	 STAR_val.json
features.json	STAR_train.json  ZeroShot_from_yesterday.ipynb


In [100]:
import json
with open("data/star/STAR_train.json", "r") as file:
    original_data = json.load(file)

In [101]:
original_data[0]

{'question_id': 'Interaction_T1_4',
 'video_id': 'TJZ0P',
 'start': 7.7,
 'end': 15.7,
 'question': 'Which object was eaten by the person?',
 'answer': 'The sandwich.',
 'question_program': [{'function': 'Situations', 'value_input': []},
  {'function': 'Actions', 'value_input': []},
  {'function': 'Filter_Actions_with_Verb', 'value_input': ['eat']},
  {'function': 'Unique', 'value_input': []},
  {'function': 'Query_Objs', 'value_input': []}],
 'choices': [{'choice_id': 0,
   'choice': 'The sandwich.',
   'choice_program': [{'function': 'Equal', 'value_input': ['sandwich']}]},
  {'choice_id': 1,
   'choice': 'The medicine.',
   'choice_program': [{'function': 'Equal', 'value_input': ['medicine']}]},
  {'choice_id': 2,
   'choice': 'The blanket.',
   'choice_program': [{'function': 'Equal', 'value_input': ['blanket']}]},
  {'choice_id': 3,
   'choice': 'The box.',
   'choice_program': [{'function': 'Equal', 'value_input': ['box']}]}],
 'situations': {'000245': {'rel_pairs': [['o000', 'o0

In [20]:
def normalize(text):
    return text.lower().strip()



In [21]:
def find_matching_original(parsed_set, original_data):
    parsed_question = normalize(parsed_set['Question'])
    parsed_choices = sorted([normalize(choice) for choice in parsed_set['Choices']])

    for original in original_data:
        original_question = normalize(original['question'])
        original_choices = sorted([normalize(choice['choice']) for choice in original['choices']])
        
        if parsed_question == original_question and parsed_choices == original_choices:
            return {
                'question_id': original['question_id'],
                'video_id': original['video_id'],
                'correct_answer': original['answer']
            }
    return None  

In [22]:
for parsed_set in parsed_data:
    match = find_matching_original(parsed_set, original_data)
    if match:
        parsed_set.update(match)

KeyboardInterrupt: 

In [ ]:
print(parsed_data[:10])

In [ ]:
parsed_data[0]

In [ ]:
original_data[0]

In [ ]:
[x for x in original_data if x['question'] == 'Which object was tidied up by the person?']

In [ ]:
for x in parsed_data[50]:
    for 

In [ ]:
parsed_data[50]

In [ ]:
original_data[49]['choices'][0]['choice_program'][0]['value_input']

In [ ]:
original_data[49]['choices'][0]['choice']

In [ ]:
[p[4:].lower() for p in parsed_data[49]['Choices']]

In [ ]:
for p_d in parsed_data:
    choices = p_d['Choices']
    new_choices = []
    for choice in choices:
        new_choices.append(choice[4:].lower())
    p_d['Question'] = p_d['Question'].lower()
    p_d['Choices'] = new_choices

In [ ]:
parsed_data[:10]

In [ ]:
test_data = parsed_data[10]

In [ ]:
test_data['Choices']

In [ ]:
[x['choice'].lower() for x in original_data[0]['choices']]

In [ ]:
set([x['choice'].lower() for x in original_data[0]['choices']])

In [ ]:
['the sandwich.', 'the medicine.', 'the blanket.', 'the box.'] == ['the sandwich.', 'the medicine.', 'the blanket.', 'the box.'][::-1]

In [ ]:
set(['the sandwich.', 'the medicine.', 'the blanket.', 'the box.']) == set(['the sandwich.', 'the medicine.', 'the blanket.', 'the box.'][::-1])

In [ ]:
test_data

In [ ]:
parsed_data[2]

In [ ]:
import re

def parse_data(lines):
    results = []
    current = {}
    choices = []
    collecting_choices = False  

    for line in lines:
        line = line.strip()
        if line.startswith('Instruction:'):
            if current:
                current['Choices'] = choices
                results.append(current)
                choices = []
            current = {'Instruction': line[len('Instruction:'):].strip()}
            collecting_choices = False  
        elif line.startswith('Question:'):
            current['Question'] = line[len('Question:'):].strip()
        elif line.startswith('Choices:'):
            collecting_choices = True  
        elif line.startswith('(') and collecting_choices:
            formatted_choice = re.sub(r'^\([A-D]\)\s*', '', line)
            choices.append(formatted_choice)
        elif line.startswith('Answer:'):
            match = re.search(r'\(.\)', line)
            if match:
                current['Answer'] = match.group(0)
            collecting_choices = False 
        elif line.startswith('### Explanation:'):

            current['Choices'] = choices
            results.append(current)
            current = {}  
            choices = []
    if current:
        current['Choices'] = choices
        results.append(current)

    return results



In [99]:
parsed_data = parse_data(raw_data)
parsed_data[0]


{'Instruction': 'Predict the answer based on the question.',
 'Question': 'Which object was tidied up by the person?',
 'Answer': '(A)',
 'Choices': ['(A) The closet/cabinet.',
  '(B) The blanket.',
  '(C) The clothes.',
  '(D) The table.']}

In [ ]:
parsed_data[:5]

In [ ]:
raw_data[:50]

In [ ]:
parsed_data[:5]

In [ ]:
original_data[:5]

In [ ]:
test_data

In [ ]:
original_data[2]

In [ ]:
[x['choice'].lower() for x in original_data[0]['choices']]

In [ ]:
test_data['Choices']

In [ ]:
unmatched_entries = [] 

for parsed_data_ele in parsed_data:
    match_found = False  
    for og_data in original_data:
        if parsed_data_ele['Question'].lower() == og_data['question'].lower():
            og_choices = set([x['choice'].lower() for x in og_data['choices']])
            parsed_choices = set([choice.lower() for choice in parsed_data_ele['Choices']])
            
            if og_choices == parsed_choices:
                parsed_data_ele['question_id'] = og_data['question_id']
                parsed_data_ele['answer'] = og_data['answer']
                parsed_data_ele['video_id'] = og_data['video_id']
                match_found = True
                break  
    
    if not match_found:
        unmatched_entries.append(parsed_data_ele)



In [ ]:
if unmatched_entries:
    print("No matches found for the following entries:")
    for entry in unmatched_entries:
        print(entry)
else:
    print("All entries were successfully matched.")

In [ ]:

potential_matches = {}
no_matches_found = []

for unmatched_entry in unmatched_entries:
    question_matches = []  
    for og_data in original_data:
        if unmatched_entry['Question'].lower() == og_data['question'].lower():
            question_matches.append(og_data['question_id'])  

    if question_matches:
        potential_matches[unmatched_entry['Question']] = question_matches
    else:
        no_matches_found.append(unmatched_entry)  




In [ ]:
if potential_matches:
    print("Potential matches based on questions were found for:")
    for question, matches in potential_matches.items():
        print(f"Question: {question} matched with Question IDs: {matches}")
else:
    print("No potential question matches found.")



In [ ]:
if no_matches_found:
    print("No matches found at all for the following entries:")
    for entry in no_matches_found:
        print(entry)
else:
    print("All unmatched entries found potential question matches.")

In [ ]:
[x for x in original_data if x['question'] == "What did the person do while they were lying on the clothes?"]

In [23]:
import pandas as pd

parsed_df = pd.DataFrame(parsed_data)
original_df = pd.DataFrame(original_data)

In [24]:
parsed_df['Question'] = parsed_df['Question'].str.lower()
original_df['question'] = original_df['question'].str.lower()

In [25]:
merged_df = pd.merge(parsed_df, original_df, left_on='Question', right_on='question', how='left')


In [26]:
matched_df = merged_df[merged_df['question_id'].notna()]
unmatched_df = merged_df[merged_df['question_id'].isna()]

In [27]:
if not unmatched_df.empty:
    print("No matches found at all for the following entries:")
    print(unmatched_df[['Instruction', 'Question']])
else:
    print("All unmatched entries found potential question matches.")

No matches found at all for the following entries:
                                       Instruction  \
1168074  Predict the answer based on the question.   
1178051  Predict the answer based on the question.   
1178052  Predict the answer based on the question.   
1178053  Predict the answer based on the question.   
1178056  Predict the answer based on the question.   
...                                            ...   
1297459  Predict the answer based on the question.   
1297460  Predict the answer based on the question.   
1297507  Predict the answer based on the question.   
1297534  Predict the answer based on the question.   
1297542  Predict the answer based on the question.   

                                                  Question  
1168074  what did the person do while they were lying o...  
1178051  what did the person do while they were covered...  
1178052  what did the person do while they were touchin...  
1178053  what did the person do while they were touchin.

In [28]:
if not matched_df.empty:
    print("Entries with potential matches:")
    print(matched_df[['Instruction', 'Question', 'question_id']])

Entries with potential matches:
                                       Instruction  \
0        Predict the answer based on the question.   
1        Predict the answer based on the question.   
2        Predict the answer based on the question.   
3        Predict the answer based on the question.   
4        Predict the answer based on the question.   
...                                            ...   
1297552  Predict the answer based on the question.   
1297553  Predict the answer based on the question.   
1297554  Predict the answer based on the question.   
1297555  Predict the answer based on the question.   
1297556  Predict the answer based on the question.   

                                                  Question         question_id  
0                which object was tidied up by the person?   Interaction_T1_20  
1                which object was tidied up by the person?   Interaction_T1_53  
2                which object was tidied up by the person?  Interaction_T1_1

In [29]:
unmatched_df

,Instruction,Question,Answer,Choices,question_id,video_id,start,end,question,answer,question_program,choices,situations
1168074,Predict the answer based on the question.,what did the person do while they were lying o...,(A),"[(A) Took the clothes., (B) Tidied up the clot...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1178051,Predict the answer based on the question.,what did the person do while they were covered...,(B),"[(A) Lied on the floor., (B) Took the paper/no...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1178052,Predict the answer based on the question.,what did the person do while they were touchin...,(A),"[(A) Washed the cup/glass/bottle., (B) Lied on...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1178053,Predict the answer based on the question.,what did the person do while they were touchin...,(A),"[(A) Lied on the floor., (B) Took the shoe., (...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1178056,Predict the answer based on the question.,what did the person do while they were standin...,(B),"[(A) Opened the bag., (B) Threw the bag., (C) ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297459,Predict the answer based on the question.,what did the person do to the medicine before ...,(A),"[(A) Lied on., (B) Threw., (C) Sat on., (D) Ate.]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297460,Predict the answer based on the question.,what did the person do to the medicine before ...,(C),"[(A) Tidied up., (B) Ate., (C) Washed., (D) Cl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297507,Predict the answer based on the question.,what did the person do to the laptop before cl...,(B),"[(A) Opened., (B) Closed., (C) Took., (D) Put ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297534,Predict the answer based on the question.,what did the person do to the box before openi...,(B),"[(A) Put down., (B) Sat at., (C) Threw., (D) T...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
merged_df

,Instruction,Question,Answer,Choices,question_id,video_id,start,end,question,answer,question_program,choices,situations
0,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...",Interaction_T1_20,GM3UK,0.0,9.9,which object was tidied up by the person?,The towel.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The towel.', 'cho...","{'000017': {'rel_pairs': [['o000', 'o006'], ['..."
1,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...",Interaction_T1_53,SJ51G,5.9,13.2,which object was tidied up by the person?,The blanket.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The blanket.', 'c...","{'000041': {'rel_pairs': [['o000', 'o027'], ['..."
2,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...",Interaction_T1_142,EURF2,13.9,23.3,which object was tidied up by the person?,The blanket.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The blanket.', 'c...","{'000417': {'rel_pairs': [['o000', 'o004'], ['..."
3,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...",Interaction_T1_159,165K2,0.4,7.0,which object was tidied up by the person?,The table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The clothes.', 'c...","{'000033': {'rel_pairs': [['o000', 'o005'], ['..."
4,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...",Interaction_T1_281,2P7A9,28.9,37.0,which object was tidied up by the person?,The table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The broom.', 'cho...","{'000872': {'rel_pairs': [['o000', 'o006'], ['..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297552,Predict the answer based on the question.,what did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ...",Sequence_T6_1781,LWEPB,11.4,26.3,what did the person do to the phone/camera bef...,Put down.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Took.', 'choice_p...","{'000292': {'rel_pairs': [['o000', 'o009'], ['..."
1297553,Predict the answer based on the question.,what did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ...",Sequence_T6_1906,5GK8W,13.3,34.4,what did the person do to the phone/camera bef...,Put down.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Sat at.', 'choice...","{'000951': {'rel_pairs': [['o000', 'o012'], ['..."
1297554,Predict the answer based on the question.,what did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ...",Sequence_T6_4560,PB7PQ,7.3,21.3,what did the person do to the phone/camera bef...,Took.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Threw.', 'choice_...","{'000276': {'rel_pairs': [['o000', 'o020'], ['..."
1297555,Predict the answer based on the question.,what did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ...",Sequence_T6_4915,F1VEE,19.2,32.0,what did the person do to the phone/camera bef...,Took.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Took.', 'choice_p...","{'000611': {'rel_pairs': [['o000', 'o014'], ['..."


In [31]:
merged_df.rename(columns={'Answer': 'generated_answer', 'answer': 'correct_answer'}, inplace=True)


In [32]:
len(parsed_data)

5789

In [33]:
original_df.shape

(45731, 9)

In [35]:
parsed_df.shape

(5789, 4)

In [37]:
matched_df.shape

(1297246, 13)

In [38]:
unmatched_df.shape

(311, 13)

In [48]:
parsed_df = pd.DataFrame(parsed_data)
original_df = pd.DataFrame(original_data)

parsed_df['Question'] = parsed_df['Question'].str.lower()
original_df['question'] = original_df['question'].str.lower()

In [49]:
parsed_df.rename(columns={'Answer': 'generated_answer'}, inplace= True)

In [50]:
parsed_df

,Instruction,Question,generated_answer,Choices
0,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C..."
1,Predict the answer based on the question.,which object was tidied up by the person?,(B),"[(A) The blanket., (B) The table., (C) The clo..."
2,Predict the answer based on the question.,which object was thrown by the person?,(B),"[(A) The pillow., (B) The bag., (C) The clothe..."
3,Predict the answer based on the question.,which object was put down by the person?,(B),"[(A) The food., (B) The shoe., (C) The blanket..."
4,Predict the answer based on the question.,which object was tidied up by the person?,(B),"[(A) The broom., (B) The closet/cabinet., (C) ..."
...,...,...,...,...
5784,Predict the answer based on the question.,what did the person do to the box before takin...,(B),"[(A) Opened., (B) Closed., (C) Threw., (D) Put..."
5785,Predict the answer based on the question.,what did the person do to the clothes before e...,(A),"[(A) Threw., (B) Sat on., (C) Tidied up., (D) ..."
5786,Predict the answer based on the question.,what did the person do to the book before putt...,(B),"[(A) Took., (B) Closed., (C) Threw., (D) Sat on.]"
5787,Predict the answer based on the question.,what did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ..."


In [51]:
original_df.rename(columns={'answer': 'correct_answer'}, inplace= True)

In [52]:
original_df

,question_id,video_id,start,end,question,correct_answer,question_program,choices,situations
0,Interaction_T1_4,TJZ0P,7.7,15.7,which object was eaten by the person?,The sandwich.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The sandwich.', '...","{'000245': {'rel_pairs': [['o000', 'o006'], ['..."
1,Interaction_T1_5,TJZ0P,6.7,11.5,which object was closed by the person?,The book.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The book.', 'choi...","{'000203': {'rel_pairs': [['o000', 'o006'], ['..."
2,Interaction_T1_7,B82GJ,17.7,22.5,which object was taken by the person?,The towel.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000546': {'rel_pairs': [['o000', 'o028'], ['..."
3,Interaction_T1_8,DUZDL,1.6,7.8,which object was sat on by the person?,The bed.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The sofa/couch.',...","{'000181': {'rel_pairs': [['o000', 'o003'], ['..."
4,Interaction_T1_9,DUZDL,31.2,40.1,which object was put down by the person?,The shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000830': {'rel_pairs': [['o000', 'o012'], ['..."
...,...,...,...,...,...,...,...,...,...
45726,Feasibility_T6_1464,LTC5Y,1.1,10.8,what is the person able to do after putting do...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Tidy up the table...","{'000041': {'rel_pairs': [['o000', 'o015'], ['..."
45727,Feasibility_T6_1467,B71ZJ,4.0,8.2,what is the person able to do after putting do...,Take the shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Sit at the table....","{'000112': {'rel_pairs': [['o000', 'o015'], ['..."
45728,Feasibility_T6_1469,FPWD0,0.1,7.8,what is the person able to do after opening th...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Throw the shoe.',...","{'000334': {'rel_pairs': [['o000', 'o008'], ['..."
45729,Feasibility_T6_1470,FPWD0,0.0,4.0,what is the person able to do after walking th...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Wash the table.',...","{'000334': {'rel_pairs': [['o000', 'o008'], ['..."


In [53]:
import pandas as pd



def create_choice_dict(choices):
    choice_dict = {}
    for choice in choices:
        key, value = choice.split(' ', 1)
        value = value.strip()
        choice_dict[key] = value
    return choice_dict

parsed_df['choice_dict'] = parsed_df['Choices'].apply(create_choice_dict)

print(parsed_df[['Instruction', 'Question', 'generated_answer', 'Choices', 'choice_dict']].head())


                                 Instruction  \
0  Predict the answer based on the question.   
1  Predict the answer based on the question.   
2  Predict the answer based on the question.   
3  Predict the answer based on the question.   
4  Predict the answer based on the question.   

                                    Question generated_answer  \
0  which object was tidied up by the person?              (A)   
1  which object was tidied up by the person?              (B)   
2     which object was thrown by the person?              (B)   
3   which object was put down by the person?              (B)   
4  which object was tidied up by the person?              (B)   

                                             Choices  \
0  [(A) The closet/cabinet., (B) The blanket., (C...   
1  [(A) The blanket., (B) The table., (C) The clo...   
2  [(A) The pillow., (B) The bag., (C) The clothe...   
3  [(A) The food., (B) The shoe., (C) The blanket...   
4  [(A) The broom., (B) The closet/cabin

In [54]:
parsed_df

,Instruction,Question,generated_answer,Choices,choice_dict
0,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla..."
1,Predict the answer based on the question.,which object was tidied up by the person?,(B),"[(A) The blanket., (B) The table., (C) The clo...","{'(A)': 'The blanket.', '(B)': 'The table.', '..."
2,Predict the answer based on the question.,which object was thrown by the person?,(B),"[(A) The pillow., (B) The bag., (C) The clothe...","{'(A)': 'The pillow.', '(B)': 'The bag.', '(C)..."
3,Predict the answer based on the question.,which object was put down by the person?,(B),"[(A) The food., (B) The shoe., (C) The blanket...","{'(A)': 'The food.', '(B)': 'The shoe.', '(C)'..."
4,Predict the answer based on the question.,which object was tidied up by the person?,(B),"[(A) The broom., (B) The closet/cabinet., (C) ...","{'(A)': 'The broom.', '(B)': 'The closet/cabin..."
...,...,...,...,...,...
5784,Predict the answer based on the question.,what did the person do to the box before takin...,(B),"[(A) Opened., (B) Closed., (C) Threw., (D) Put...","{'(A)': 'Opened.', '(B)': 'Closed.', '(C)': 'T..."
5785,Predict the answer based on the question.,what did the person do to the clothes before e...,(A),"[(A) Threw., (B) Sat on., (C) Tidied up., (D) ...","{'(A)': 'Threw.', '(B)': 'Sat on.', '(C)': 'Ti..."
5786,Predict the answer based on the question.,what did the person do to the book before putt...,(B),"[(A) Took., (B) Closed., (C) Threw., (D) Sat on.]","{'(A)': 'Took.', '(B)': 'Closed.', '(C)': 'Thr..."
5787,Predict the answer based on the question.,what did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ...","{'(A)': 'Took.', '(B)': 'Sat on.', '(C)': 'Sat..."


In [55]:
original_df

,question_id,video_id,start,end,question,correct_answer,question_program,choices,situations
0,Interaction_T1_4,TJZ0P,7.7,15.7,which object was eaten by the person?,The sandwich.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The sandwich.', '...","{'000245': {'rel_pairs': [['o000', 'o006'], ['..."
1,Interaction_T1_5,TJZ0P,6.7,11.5,which object was closed by the person?,The book.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The book.', 'choi...","{'000203': {'rel_pairs': [['o000', 'o006'], ['..."
2,Interaction_T1_7,B82GJ,17.7,22.5,which object was taken by the person?,The towel.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000546': {'rel_pairs': [['o000', 'o028'], ['..."
3,Interaction_T1_8,DUZDL,1.6,7.8,which object was sat on by the person?,The bed.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The sofa/couch.',...","{'000181': {'rel_pairs': [['o000', 'o003'], ['..."
4,Interaction_T1_9,DUZDL,31.2,40.1,which object was put down by the person?,The shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000830': {'rel_pairs': [['o000', 'o012'], ['..."
...,...,...,...,...,...,...,...,...,...
45726,Feasibility_T6_1464,LTC5Y,1.1,10.8,what is the person able to do after putting do...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Tidy up the table...","{'000041': {'rel_pairs': [['o000', 'o015'], ['..."
45727,Feasibility_T6_1467,B71ZJ,4.0,8.2,what is the person able to do after putting do...,Take the shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Sit at the table....","{'000112': {'rel_pairs': [['o000', 'o015'], ['..."
45728,Feasibility_T6_1469,FPWD0,0.1,7.8,what is the person able to do after opening th...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Throw the shoe.',...","{'000334': {'rel_pairs': [['o000', 'o008'], ['..."
45729,Feasibility_T6_1470,FPWD0,0.0,4.0,what is the person able to do after walking th...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Wash the table.',...","{'000334': {'rel_pairs': [['o000', 'o008'], ['..."


In [56]:
def extract_choice_set(choices):
    return set(choice_dict['choice'] for choice_dict in choices)

original_df['choice_set'] = original_df['choices'].apply(extract_choice_set)

In [57]:
print(original_df[['question_id', 'video_id', 'question', 'correct_answer', 'choices', 'choice_set']].head())


        question_id video_id                                  question  \
0  Interaction_T1_4    TJZ0P     which object was eaten by the person?   
1  Interaction_T1_5    TJZ0P    which object was closed by the person?   
2  Interaction_T1_7    B82GJ     which object was taken by the person?   
3  Interaction_T1_8    DUZDL    which object was sat on by the person?   
4  Interaction_T1_9    DUZDL  which object was put down by the person?   

  correct_answer                                            choices  \
0  The sandwich.  [{'choice_id': 0, 'choice': 'The sandwich.', '...   
1      The book.  [{'choice_id': 0, 'choice': 'The book.', 'choi...   
2     The towel.  [{'choice_id': 0, 'choice': 'The food.', 'choi...   
3       The bed.  [{'choice_id': 0, 'choice': 'The sofa/couch.',...   
4      The shoe.  [{'choice_id': 0, 'choice': 'The food.', 'choi...   

                                          choice_set  
0  {The blanket., The box., The sandwich., The me...  
1  {The book., The

In [58]:
original_df

,question_id,video_id,start,end,question,correct_answer,question_program,choices,situations,choice_set
0,Interaction_T1_4,TJZ0P,7.7,15.7,which object was eaten by the person?,The sandwich.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The sandwich.', '...","{'000245': {'rel_pairs': [['o000', 'o006'], ['...","{The blanket., The box., The sandwich., The me..."
1,Interaction_T1_5,TJZ0P,6.7,11.5,which object was closed by the person?,The book.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The book.', 'choi...","{'000203': {'rel_pairs': [['o000', 'o006'], ['...","{The book., The closet/cabinet., The window., ..."
2,Interaction_T1_7,B82GJ,17.7,22.5,which object was taken by the person?,The towel.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000546': {'rel_pairs': [['o000', 'o028'], ['...","{The towel., The laptop., The broom., The food.}"
3,Interaction_T1_8,DUZDL,1.6,7.8,which object was sat on by the person?,The bed.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The sofa/couch.',...","{'000181': {'rel_pairs': [['o000', 'o003'], ['...","{The floor., The sofa/couch., The bed., The ta..."
4,Interaction_T1_9,DUZDL,31.2,40.1,which object was put down by the person?,The shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000830': {'rel_pairs': [['o000', 'o012'], ['...","{The clothes., The broom., The food., The shoe.}"
...,...,...,...,...,...,...,...,...,...,...
45726,Feasibility_T6_1464,LTC5Y,1.1,10.8,what is the person able to do after putting do...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Tidy up the table...","{'000041': {'rel_pairs': [['o000', 'o015'], ['...","{Sit at the table., Close the laptop., Throw t..."
45727,Feasibility_T6_1467,B71ZJ,4.0,8.2,what is the person able to do after putting do...,Take the shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Sit at the table....","{'000112': {'rel_pairs': [['o000', 'o015'], ['...","{Sit at the table., Sit on the floor., Lie on ..."
45728,Feasibility_T6_1469,FPWD0,0.1,7.8,what is the person able to do after opening th...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Throw the shoe.',...","{'000334': {'rel_pairs': [['o000', 'o008'], ['...","{Sit at the table., Throw the shoe., Wash the ..."
45729,Feasibility_T6_1470,FPWD0,0.0,4.0,what is the person able to do after walking th...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Wash the table.',...","{'000334': {'rel_pairs': [['o000', 'o008'], ['...","{Sit at the table., Wash the table., Put down ..."


In [59]:
def extract_set_from_dict(choice_dict):
    return set(choice_dict.values())

parsed_df['choice_set'] = parsed_df['choice_dict'].apply(extract_set_from_dict)


In [60]:
parsed_df

,Instruction,Question,generated_answer,Choices,choice_dict,choice_set
0,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table...."
1,Predict the answer based on the question.,which object was tidied up by the person?,(B),"[(A) The blanket., (B) The table., (C) The clo...","{'(A)': 'The blanket.', '(B)': 'The table.', '...","{The blanket., The closet/cabinet., The table...."
2,Predict the answer based on the question.,which object was thrown by the person?,(B),"[(A) The pillow., (B) The bag., (C) The clothe...","{'(A)': 'The pillow.', '(B)': 'The bag.', '(C)...","{The clothes., The pillow., The bag., The box.}"
3,Predict the answer based on the question.,which object was put down by the person?,(B),"[(A) The food., (B) The shoe., (C) The blanket...","{'(A)': 'The food.', '(B)': 'The shoe.', '(C)'...","{The blanket., The sandwich., The food., The s..."
4,Predict the answer based on the question.,which object was tidied up by the person?,(B),"[(A) The broom., (B) The closet/cabinet., (C) ...","{'(A)': 'The broom.', '(B)': 'The closet/cabin...","{The blanket., The closet/cabinet., The broom...."
...,...,...,...,...,...,...
5784,Predict the answer based on the question.,what did the person do to the box before takin...,(B),"[(A) Opened., (B) Closed., (C) Threw., (D) Put...","{'(A)': 'Opened.', '(B)': 'Closed.', '(C)': 'T...","{Threw., Put down., Opened., Closed.}"
5785,Predict the answer based on the question.,what did the person do to the clothes before e...,(A),"[(A) Threw., (B) Sat on., (C) Tidied up., (D) ...","{'(A)': 'Threw.', '(B)': 'Sat on.', '(C)': 'Ti...","{Put down., Sat on., Threw., Tidied up.}"
5786,Predict the answer based on the question.,what did the person do to the book before putt...,(B),"[(A) Took., (B) Closed., (C) Threw., (D) Sat on.]","{'(A)': 'Took.', '(B)': 'Closed.', '(C)': 'Thr...","{Sat on., Threw., Closed., Took.}"
5787,Predict the answer based on the question.,what did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ...","{'(A)': 'Took.', '(B)': 'Sat on.', '(C)': 'Sat...","{Sat at., Put down., Sat on., Took.}"


In [61]:
parsed_df['Question'] = parsed_df['Question'].str.lower().str.strip()
original_df['question'] = original_df['question'].str.lower().str.strip()

In [62]:
merged_df = pd.merge(parsed_df, original_df, left_on=['Question', 'choice_set'], right_on=['question', 'choice_set'], how='left')


TypeError: unhashable type: 'set'

In [63]:
parsed_df['choice_frozenset'] = parsed_df['choice_set'].apply(frozenset)


In [64]:
original_df['choice_frozenset'] = original_df['choice_set'].apply(frozenset)

In [65]:
merged_df = pd.merge(parsed_df, original_df, left_on=['Question', 'choice_frozenset'], right_on=['question', 'choice_frozenset'], how='left')


In [66]:
unmatched_df = merged_df[merged_df['question_id'].isna()]


In [69]:
print("Matched Data Sample:")
print(merged_df.dropna(subset=['question_id']))

Matched Data Sample:
                                     Instruction  \
0      Predict the answer based on the question.   
1      Predict the answer based on the question.   
2      Predict the answer based on the question.   
3      Predict the answer based on the question.   
4      Predict the answer based on the question.   
...                                          ...   
69946  Predict the answer based on the question.   
69947  Predict the answer based on the question.   
69948  Predict the answer based on the question.   
69951  Predict the answer based on the question.   
69957  Predict the answer based on the question.   

                                                Question generated_answer  \
0              which object was tidied up by the person?              (A)   
1              which object was tidied up by the person?              (A)   
2              which object was tidied up by the person?              (A)   
3              which object was tidied up by t

In [70]:
merged_df.shape

(69962, 17)

In [71]:
print("\nUnmatched Data Sample:")
print(unmatched_df[['Instruction', 'Question', 'choice_frozenset']])


Unmatched Data Sample:
                                     Instruction  \
56     Predict the answer based on the question.   
67     Predict the answer based on the question.   
324    Predict the answer based on the question.   
472    Predict the answer based on the question.   
473    Predict the answer based on the question.   
...                                          ...   
69956  Predict the answer based on the question.   
69958  Predict the answer based on the question.   
69959  Predict the answer based on the question.   
69960  Predict the answer based on the question.   
69961  Predict the answer based on the question.   

                                                Question  \
56              which object was put down by the person?   
67                which object was thrown by the person?   
324                which object was taken by the person?   
472             which object was put down by the person?   
473             which object was put down by the pe

In [72]:
print(f"\nTotal entries in parsed_df: {len(parsed_df)}")
print(f"Total matched entries: {len(merged_df) - len(unmatched_df)}")
print(f"Total unmatched entries: {len(unmatched_df)}")


Total entries in parsed_df: 5789
Total matched entries: 66240
Total unmatched entries: 3722


In [73]:
original_df

,question_id,video_id,start,end,question,correct_answer,question_program,choices,situations,choice_set,choice_frozenset
0,Interaction_T1_4,TJZ0P,7.7,15.7,which object was eaten by the person?,The sandwich.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The sandwich.', '...","{'000245': {'rel_pairs': [['o000', 'o006'], ['...","{The blanket., The box., The sandwich., The me...","(The blanket., The box., The sandwich., The me..."
1,Interaction_T1_5,TJZ0P,6.7,11.5,which object was closed by the person?,The book.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The book.', 'choi...","{'000203': {'rel_pairs': [['o000', 'o006'], ['...","{The book., The closet/cabinet., The window., ...","(The book., The closet/cabinet., The window., ..."
2,Interaction_T1_7,B82GJ,17.7,22.5,which object was taken by the person?,The towel.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000546': {'rel_pairs': [['o000', 'o028'], ['...","{The towel., The laptop., The broom., The food.}","(The towel., The laptop., The broom., The food.)"
3,Interaction_T1_8,DUZDL,1.6,7.8,which object was sat on by the person?,The bed.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The sofa/couch.',...","{'000181': {'rel_pairs': [['o000', 'o003'], ['...","{The floor., The sofa/couch., The bed., The ta...","(The floor., The sofa/couch., The bed., The ta..."
4,Interaction_T1_9,DUZDL,31.2,40.1,which object was put down by the person?,The shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000830': {'rel_pairs': [['o000', 'o012'], ['...","{The clothes., The broom., The food., The shoe.}","(The clothes., The broom., The food., The shoe.)"
...,...,...,...,...,...,...,...,...,...,...,...
45726,Feasibility_T6_1464,LTC5Y,1.1,10.8,what is the person able to do after putting do...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Tidy up the table...","{'000041': {'rel_pairs': [['o000', 'o015'], ['...","{Sit at the table., Close the laptop., Throw t...","(Sit at the table., Close the laptop., Throw t..."
45727,Feasibility_T6_1467,B71ZJ,4.0,8.2,what is the person able to do after putting do...,Take the shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Sit at the table....","{'000112': {'rel_pairs': [['o000', 'o015'], ['...","{Sit at the table., Sit on the floor., Lie on ...","(Sit at the table., Sit on the floor., Lie on ..."
45728,Feasibility_T6_1469,FPWD0,0.1,7.8,what is the person able to do after opening th...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Throw the shoe.',...","{'000334': {'rel_pairs': [['o000', 'o008'], ['...","{Sit at the table., Throw the shoe., Wash the ...","(Sit at the table., Throw the shoe., Wash the ..."
45729,Feasibility_T6_1470,FPWD0,0.0,4.0,what is the person able to do after walking th...,Sit at the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Wash the table.',...","{'000334': {'rel_pairs': [['o000', 'o008'], ['...","{Sit at the table., Wash the table., Put down ...","(Sit at the table., Wash the table., Put down ..."


In [74]:
parsed_df['choice_list'] = parsed_df['choice_dict'].apply(lambda d: [d[k] for k in sorted(d.keys(), key=lambda x: x)])



In [75]:
original_df['choice_list'] = original_df['choices'].apply(lambda choices: [choice['choice'] for choice in sorted(choices, key=lambda x: x['choice_id'])])


In [76]:
merged_df = pd.merge(parsed_df, original_df, left_on=['Question', 'choice_list'], right_on=['question', 'choice_list'], how='left')


TypeError: unhashable type: 'list'

In [77]:
parsed_df['choice_string'] = parsed_df['choice_list'].apply(lambda x: '||'.join(x))


In [78]:
original_df['choice_string'] = original_df['choice_list'].apply(lambda x: '||'.join(x))


In [79]:
merged_df = pd.merge(parsed_df, original_df, left_on=['Question', 'choice_string'], right_on=['question', 'choice_string'], how='left')


In [80]:
unmatched_df = merged_df[merged_df['question_id'].isna()]


In [81]:
print(f"\nTotal entries in parsed_df: {len(parsed_df)}")
print(f"Total matched entries: {len(merged_df) - len(unmatched_df)}")
print(f"Total unmatched entries: {len(unmatched_df)}")



Total entries in parsed_df: 5789
Total matched entries: 2793
Total unmatched entries: 4887


In [82]:
matched_df = merged_df.dropna(subset=['question_id'])


In [83]:
matched_df

,Instruction,Question,generated_answer,Choices,choice_dict,choice_set_x,choice_frozenset_x,choice_list_x,choice_string,question_id,...,start,end,question,correct_answer,question_program,choices,situations,choice_set_y,choice_frozenset_y,choice_list_y
0,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_6646,...,24.4,32.0,which object was tidied up by the person?,The table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000628': {'rel_pairs': [['o000', 'o006'], ['...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe..."
1,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_6998,...,15.1,21.9,which object was tidied up by the person?,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000461': {'rel_pairs': [['o000', 'o019'], ['...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe..."
2,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_8120,...,0.0,6.6,which object was tidied up by the person?,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000021': {'rel_pairs': [['o000', 'o019'], ['...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe..."
3,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_8501,...,9.9,18.9,which object was tidied up by the person?,The blanket.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000486': {'rel_pairs': [['o000', 'o004'], ['...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe..."
4,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_9106,...,2.5,10.8,which object was tidied up by the person?,The table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_i

In [84]:
def find_choice_label(choices, correct_answer):
    choices_df = pd.DataFrame(choices)
    match = choices_df[choices_df['choice'] == correct_answer]
    if not match.empty:
        return f"({chr(65 + match.iloc[0]['choice_id'])})"  
    else:
        return None  

In [85]:
matched_df['matched_choice'] = matched_df.apply(lambda x: find_choice_label(x['choices'], x['correct_answer']), axis=1)


/tmp/ipykernel_546068/2905841366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df['matched_choice'] = matched_df.apply(lambda x: find_choice_label(x['choices'], x['correct_answer']), axis=1)


In [86]:
matched_df['is_correct'] = matched_df['generated_answer'] == matched_df['matched_choice']


/tmp/ipykernel_546068/558698933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df['is_correct'] = matched_df['generated_answer'] == matched_df['matched_choice']


In [87]:
print(matched_df[['Instruction', 'Question', 'generated_answer', 'correct_answer', 'matched_choice', 'is_correct']])


                                    Instruction  \
0     Predict the answer based on the question.   
1     Predict the answer based on the question.   
2     Predict the answer based on the question.   
3     Predict the answer based on the question.   
4     Predict the answer based on the question.   
...                                         ...   
7565  Predict the answer based on the question.   
7610  Predict the answer based on the question.   
7619  Predict the answer based on the question.   
7656  Predict the answer based on the question.   
7666  Predict the answer based on the question.   

                                               Question generated_answer  \
0             which object was tidied up by the person?              (A)   
1             which object was tidied up by the person?              (A)   
2             which object was tidied up by the person?              (A)   
3             which object was tidied up by the person?              (A)   
4      

In [88]:
unmatched_answers_df = matched_df[matched_df['matched_choice'].isnull()]
if not unmatched_answers_df.empty:
    print("\nEntries where correct_answer could not be matched to any choice:")
    print(unmatched_answers_df[['Instruction', 'Question', 'correct_answer', 'choices']])
else:
    print("\nAll correct answers matched successfully to a choice.")


All correct answers matched successfully to a choice.


In [89]:
matched_df

,Instruction,Question,generated_answer,Choices,choice_dict,choice_set_x,choice_frozenset_x,choice_list_x,choice_string,question_id,...,question,correct_answer,question_program,choices,situations,choice_set_y,choice_frozenset_y,choice_list_y,matched_choice,is_correct
0,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_6646,...,which object was tidied up by the person?,The table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000628': {'rel_pairs': [['o000', 'o006'], ['...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",(D),False
1,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_6998,...,which object was tidied up by the person?,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000461': {'rel_pairs': [['o000', 'o019'], ['...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",(C),False
2,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_8120,...,which object was tidied up by the person?,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000021': {'rel_pairs': [['o000', 'o019'], ['...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",(C),False
3,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_8501,...,which object was tidied up by the person?,The blanket.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000486': {'rel_pairs': [['o000', 'o004'], ['...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",(B),False
4,Predict the answer based on the question.,which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...","{'(A)': 'The closet/cabinet.', '(B)': 'The bla...","{The blanket., The closet/cabinet., The table....","(The blanket., The closet/cabinet., The table....","[The closet/cabinet., The blanket., The clothe...",The closet/cabinet.||The blanket.||The clothes...,Interaction_T1_9106,...,which object was tidied up by the person?,The table.,"[{'function': 'Situations', 'value_input': []}...","[

In [90]:
correct_counts = matched_df['is_correct'].value_counts()


In [97]:
correct_counts

False    2113
True      680
Name: is_correct, dtype: int64

In [98]:
2113 / 2793

0.7565341926244182

# Realization: we can just use the order to match-up the questions and answers
Recognizing that we generated our data in a deterministic fashion, we can 'reverse-engineer' this, by simply following the same order

In [ ]:
import json
with open("data/star/STAR_train.json", "r") as file:
    original_data = json.load(file)


In [134]:
parsed_data = parse_data(raw_data)

In [135]:
parsed_df = pd.DataFrame(parsed_data)
original_df = pd.DataFrame(original_data)

In [136]:
parsed_df.shape

(5789, 5)

In [137]:
original_df.shape

(45731, 9)

In [138]:
parsed_df.reset_index(drop=True, inplace=True)
original_df.reset_index(drop=True, inplace=True)

In [139]:
parsed_df['question_id'] = original_df['question_id']
parsed_df['video_id'] = original_df['video_id']
parsed_df['correct_answer'] = original_df['answer']
parsed_df['choices_detail'] = original_df['choices']
parsed_df

,Instruction,Question,Answer,Choices,Explanation,question_id,video_id,correct_answer,choices_detail
0,Predict the answer based on the question.,Which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...",NaN,Interaction_T1_4,TJZ0P,The sandwich.,"[{'choice_id': 0, 'choice': 'The sandwich.', '..."
1,Predict the answer based on the question.,Which object was tidied up by the person?,(B),"[(A) The blanket., (B) The table., (C) The clo...","In the picture, we can see that the table is t...",Interaction_T1_5,TJZ0P,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi..."
2,Predict the answer based on the question.,Which object was thrown by the person?,(B),"[(A) The pillow., (B) The bag., (C) The clothe...","In the picture, we can see that a person is ho...",Interaction_T1_7,B82GJ,The towel.,"[{'choice_id': 0, 'choice': 'The food.', 'choi..."
3,Predict the answer based on the question.,Which object was put down by the person?,(B),"[(A) The food., (B) The shoe., (C) The blanket...",Choices:,Interaction_T1_8,DUZDL,The bed.,"[{'choice_id': 0, 'choice': 'The sofa/couch.',..."
4,Predict the answer based on the question.,Which object was tidied up by the person?,(B),"[(A) The broom., (B) The closet/cabinet., (C) ...",NaN,Interaction_T1_9,DUZDL,The shoe.,"[{'choice_id': 0, 'choice': 'The food.', 'choi..."
...,...,...,...,...,...,...,...,...,...
5784,Predict the answer based on the question.,What did the person do to the box before takin...,(B),"[(A) Opened., (B) Closed., (C) Threw., (D) Put...",Choices: The person took a box and put it on ...,Interaction_T2_1127,KCRQU,Opened.,"[{'choice_id': 0, 'choice': 'Opened.', 'choice..."
5785,Predict the answer based on the question.,What did the person do to the clothes before e...,(A),"[(A) Threw., (B) Sat on., (C) Tidied up., (D) ...",Choices: The person threw the dirty clothes aw...,Interaction_T2_1129,5VUT9,Put down.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_..."
5786,Predict the answer based on the question.,What did the person do to the book before putt...,(B),"[(A) Took., (B) Closed., (C) Threw., (D) Sat on.]",Choices: The person closed the notebook before...,Interaction_T2_1130,5VUT9,Opened.,"[{'choice_id': 0, 'choice': 'Closed.', 'choice..."
5787,Predict the answer based on the question.,What did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ...",Choices: The person sat on/sat at/put down/too...,Interaction_T2_1131,5VUT9,Took.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_..."


In [140]:
parsed_df.rename(columns={'Answer': 'generated_answer'}, inplace=True)
parsed_df

,Instruction,Question,generated_answer,Choices,Explanation,question_id,video_id,correct_answer,choices_detail
0,Predict the answer based on the question.,Which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...",NaN,Interaction_T1_4,TJZ0P,The sandwich.,"[{'choice_id': 0, 'choice': 'The sandwich.', '..."
1,Predict the answer based on the question.,Which object was tidied up by the person?,(B),"[(A) The blanket., (B) The table., (C) The clo...","In the picture, we can see that the table is t...",Interaction_T1_5,TJZ0P,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi..."
2,Predict the answer based on the question.,Which object was thrown by the person?,(B),"[(A) The pillow., (B) The bag., (C) The clothe...","In the picture, we can see that a person is ho...",Interaction_T1_7,B82GJ,The towel.,"[{'choice_id': 0, 'choice': 'The food.', 'choi..."
3,Predict the answer based on the question.,Which object was put down by the person?,(B),"[(A) The food., (B) The shoe., (C) The blanket...",Choices:,Interaction_T1_8,DUZDL,The bed.,"[{'choice_id': 0, 'choice': 'The sofa/couch.',..."
4,Predict the answer based on the question.,Which object was tidied up by the person?,(B),"[(A) The broom., (B) The closet/cabinet., (C) ...",NaN,Interaction_T1_9,DUZDL,The shoe.,"[{'choice_id': 0, 'choice': 'The food.', 'choi..."
...,...,...,...,...,...,...,...,...,...
5784,Predict the answer based on the question.,What did the person do to the box before takin...,(B),"[(A) Opened., (B) Closed., (C) Threw., (D) Put...",Choices: The person took a box and put it on ...,Interaction_T2_1127,KCRQU,Opened.,"[{'choice_id': 0, 'choice': 'Opened.', 'choice..."
5785,Predict the answer based on the question.,What did the person do to the clothes before e...,(A),"[(A) Threw., (B) Sat on., (C) Tidied up., (D) ...",Choices: The person threw the dirty clothes aw...,Interaction_T2_1129,5VUT9,Put down.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_..."
5786,Predict the answer based on the question.,What did the person do to the book before putt...,(B),"[(A) Took., (B) Closed., (C) Threw., (D) Sat on.]",Choices: The person closed the notebook before...,Interaction_T2_1130,5VUT9,Opened.,"[{'choice_id': 0, 'choice': 'Closed.', 'choice..."
5787,Predict the answer based on the question.,What did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ...",Choices: The person sat on/sat at/put down/too...,Interaction_T2_1131,5VUT9,Took.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_..."


In [141]:
parsed_df.generated_answer

0       (A)
1       (B)
2       (B)
3       (B)
4       (B)
       ... 
5784    (B)
5785    (A)
5786    (B)
5787    (B)
5788    (B)
Name: generated_answer, Length: 5789, dtype: object

In [142]:
parsed_df['generated_answer'].unique()

array(['(A)', '(B)', '(C)', '(D)', nan], dtype=object)

In [143]:
parsed_df['generated_answer'].dtype

dtype('O')

In [144]:
parsed_df['generated_answer'] = parsed_df['generated_answer'].fillna('(E)')

In [145]:
parsed_df['generated_answer_index'] = parsed_df['generated_answer'].apply(lambda x: ord(x.strip('()')) - ord('A'))

In [146]:
parsed_df['generated_answer_index']

0       0
1       1
2       1
3       1
4       1
       ..
5784    1
5785    0
5786    1
5787    1
5788    1
Name: generated_answer_index, Length: 5789, dtype: int64

In [147]:
parsed_df['generated_answer'].value_counts()


(B)    2745
(A)    2094
(C)     907
(D)      26
(E)      17
Name: generated_answer, dtype: int64

In [148]:
parsed_df[parsed_df['generated_answer'] == '(E)']

,Instruction,Question,generated_answer,Choices,Explanation,question_id,video_id,correct_answer,choices_detail,generated_answer_index
1711,Predict the answer based on the question.,What did the person do with the cup/glass/bottle?,(E),"[(A) Put down., (B) Took., (C) Washed., (D) Li...",NaN,Interaction_T1_3221,4VCHD,The door.,"[{'choice_id': 0, 'choice': 'The door.', 'choi...",4
1834,Predict the answer based on the question.,What did the person do while they were holding...,(E),"[(A) Took the bag., (B) Put down the bag., (C)...",NaN,Interaction_T1_3453,0TIHC,The box.,"[{'choice_id': 0, 'choice': 'The box.', 'choic...",4
1885,Predict the answer based on the question.,What did the person do while they were holding...,(E),"[(A) Closed the box., (B) Took the box., (C) P...",NaN,Interaction_T1_3549,5QW1X,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",4
2150,Predict the answer based on the question.,What did the person do while they were touchin...,(E),"[(A) Took the cup/glass/bottle., (B) Threw the...",NaN,Interaction_T1_4065,UHDEF,The towel.,"[{'choice_id': 0, 'choice': 'The broom.', 'cho...",4
2512,Predict the answer based on the question.,Which object did the person sit at after they ...,(E),"[(A) The sofa/couch., (B) The table., (C) The ...",Choices:,Interaction_T1_4722,DQGYG,The dish.,"[{'choice_id': 0, 'choice': 'The food.', 'choi...",4
2609,Predict the answer based on the question.,Which object did the person sit on after they ...,(E),"[(A) The bed., (B) The floor., (C) The medicin...",NaN,Interaction_T1_4888,AAW5D,The phone/camera.,"[{'choice_id': 0, 'choice': 'The phone/camera....",4
2925,Predict the answer based on the question.,Which object did the person sit at after they ...,(E),"[(A) The bed., (B) The floor., (C) The refrige...",Choices:,Interaction_T1_5429,EBCTL,The refrigerator.,"[{'choice_id': 0, 'choice': 'The laptop.', 'ch...",4
3502,Predict the answer based on the question.,Which object did the person sit at before they...,(E),"[(A) The floor., (B) The bed., (C) The paper/n...",Choices:,Interaction_T1_6488,CMI7U,The closet/cabinet.,"[{'choice_id': 0, 'choice': 'The closet/cabine...",4
3523,Predict the answer based on the question.,Which object did the person close before they ...,(E),"[(A) The closet/cabinet., (B) The door., (C) T...",Choices:,Interaction_T1_6516,XJH5D,The towel.,"[{'choice_id': 0, 'choice': 'The clothes.', 'c...",4
3740,Predict the answer based on the question.,Which object did the person sit at before they...,(E),"[(A) The table., (B) The sofa/couch., (C) The ...",NaN,Interaction_T1_6955,YNJ86,The book.,"[{'choice_id': 0, 'choice': 'The box.', 'choic...",4


Assuming the parsing didn't mess up, it seems fair to assess each of these as incorrect answers

In [149]:
parsed_df

,Instruction,Question,generated_answer,Choices,Explanation,question_id,video_id,correct_answer,choices_detail,generated_answer_index
0,Predict the answer based on the question.,Which object was tidied up by the person?,(A),"[(A) The closet/cabinet., (B) The blanket., (C...",NaN,Interaction_T1_4,TJZ0P,The sandwich.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",0
1,Predict the answer based on the question.,Which object was tidied up by the person?,(B),"[(A) The blanket., (B) The table., (C) The clo...","In the picture, we can see that the table is t...",Interaction_T1_5,TJZ0P,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",1
2,Predict the answer based on the question.,Which object was thrown by the person?,(B),"[(A) The pillow., (B) The bag., (C) The clothe...","In the picture, we can see that a person is ho...",Interaction_T1_7,B82GJ,The towel.,"[{'choice_id': 0, 'choice': 'The food.', 'choi...",1
3,Predict the answer based on the question.,Which object was put down by the person?,(B),"[(A) The food., (B) The shoe., (C) The blanket...",Choices:,Interaction_T1_8,DUZDL,The bed.,"[{'choice_id': 0, 'choice': 'The sofa/couch.',...",1
4,Predict the answer based on the question.,Which object was tidied up by the person?,(B),"[(A) The broom., (B) The closet/cabinet., (C) ...",NaN,Interaction_T1_9,DUZDL,The shoe.,"[{'choice_id': 0, 'choice': 'The food.', 'choi...",1
...,...,...,...,...,...,...,...,...,...,...
5784,Predict the answer based on the question.,What did the person do to the box before takin...,(B),"[(A) Opened., (B) Closed., (C) Threw., (D) Put...",Choices: The person took a box and put it on ...,Interaction_T2_1127,KCRQU,Opened.,"[{'choice_id': 0, 'choice': 'Opened.', 'choice...",1
5785,Predict the answer based on the question.,What did the person do to the clothes before e...,(A),"[(A) Threw., (B) Sat on., (C) Tidied up., (D) ...",Choices: The person threw the dirty clothes aw...,Interaction_T2_1129,5VUT9,Put down.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_...",0
5786,Predict the answer based on the question.,What did the person do to the book before putt...,(B),"[(A) Took., (B) Closed., (C) Threw., (D) Sat on.]",Choices: The person closed the notebook before...,Interaction_T2_1130,5VUT9,Opened.,"[{'choice_id': 0, 'choice': 'Closed.', 'choice...",1
5787,Predict the answer based on the question.,What did the person do to the phone/camera bef...,(B),"[(A) Took., (B) Sat on., (C) Sat at., (D) Put ...",Choices: The person sat on/sat at/put down/too...,Interaction_T2_1131,5VUT9,Took.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_...",1


In [151]:
def map_correct_answer_to_index(choices_detail, correct_answer):
    for choice in choices_detail:
        if choice['choice'] == correct_answer:
            return choice['choice_id']
    return None



In [153]:
parsed_df['correct_answer_index'] = parsed_df.apply(lambda x: map_correct_answer_to_index(x['choices_detail'], x['correct_answer']), axis=1)

In [154]:
parsed_df['correct_answer_generated'] = parsed_df['generated_answer_index'] == parsed_df['correct_answer_index']


In [156]:
parsed_df['question_type'] = parsed_df['question_id'].apply(lambda x: x.split('_')[0])


In [157]:
parsed_df['correct_answer_generated'].value_counts()

False    4384
True     1405
Name: correct_answer_generated, dtype: int64

In [158]:
parsed_df['question_type'].value_counts()

Interaction    5789
Name: question_type, dtype: int64

In [159]:
1405/4384

0.3204835766423358

In [161]:
!ls ZeroShotOutput

ZeroShotOutputIndex0.json  ZeroShotOutputIndex50.json


In [197]:
zeroShotProgressPath = "ZeroShotOutput/ZeroShotOutputIndex200.json"

In [198]:
import json
with open(zeroShotProgressPath, "r") as file:
    ZS_data = json.load(file)

In [199]:
ZS_data['Sequence_T5_3122']

{'video_id': 'APVAD',
 'correct_answer': 'Threw.',
 'choices': [{'choice_id': 0,
   'choice': 'Washed.',
   'choice_program': [{'function': 'Equal', 'value_input': ['wash']}]},
  {'choice_id': 1,
   'choice': 'Put down.',
   'choice_program': [{'function': 'Equal', 'value_input': ['put']}]},
  {'choice_id': 2,
   'choice': 'Took.',
   'choice_program': [{'function': 'Equal', 'value_input': ['take']}]},
  {'choice_id': 3,
   'choice': 'Threw.',
   'choice_program': [{'function': 'Equal', 'value_input': ['throw']}]}],
 'prompt': 'Instruction: Predict the answer based on the question.\nQuestion: What did the person do to the pillow after holding the clothes?\nChoices: \n(A) Washed.\n(B) Put down.\n(C) Took.\n(D) Threw.\nAnswer: The answer is ',
 'gen_answer': 'Instruction: Predict the answer based on the question.\nQuestion: What did the person do to the pillow after holding the clothes?\nChoices: \n(A) Washed.\n(B) Put down.\n(C) Took.\n(D) Threw.\nAnswer: The answer is  (A).\nExplanatio

In [200]:
zs_df = pd.DataFrame(ZS_data)

In [201]:
zs_df

,Sequence_T5_3122,Sequence_T1_6652,Sequence_T1_2773,Sequence_T3_311,Sequence_T5_3694,Sequence_T2_4338,Sequence_T1_2236,Prediction_T2_489,Sequence_T4_4310,Sequence_T5_1830,...,Sequence_T4_3651,Prediction_T4_1703,Sequence_T1_4662,Sequence_T5_3779,Sequence_T3_3192,Sequence_T3_3211,Sequence_T2_464,Sequence_T2_4514,Sequence_T1_4306,Prediction_T1_512
video_id,APVAD,KZODG,2XVR0,ATV2F,AK9IB,J84TZ,24XHS,52WDV,X4DOF,2WGSN,...,M1GW9,UDGRS,G87XG,VT64S,2I871,VPBIO,SS3IL,KRLT7,HNPC8,M6ERH
correct_answer,Threw.,The laptop.,The sandwich.,Put down the laptop.,Closed.,The sofa/couch.,The refrigerator.,Open.,Took the pillow.,Took.,...,Took the pillow.,The phone/camera.,The book.,Opened.,Took the bag.,Opened the bag.,The sandwich.,The clothes.,The closet/cabinet.,Sit at the table.
choices,"[{'choice_id': 0, 'choice': 'Washed.', 'choice...","[{'choice_id': 0, 'choice': 'The book.', 'choi...","[{'choice_id': 0, 'choice': 'The phone/camera....","[{'choice_id': 0, 'choice': 'Took the cup/glas...","[{'choice_id': 0, 'choice': 'Threw.', 'choice_...","[{'choice_id': 0, 'choice': 'The laptop.', 'ch...","[{'choice_id': 0, 'choice': 'The refrigerator....","[{'choice_id': 0, 'choice': 'Put down.', 'choi...","[{'choice_id': 0, 'choice': 'Threw the blanket...","[{'choice_id': 0, 'choice': 'Put down.', 'choi...",...,"[{'choice_id': 0, 'choice': 'Put down the bag....","[{'choice_id': 0, 'choice': 'The clothes.', 'c...","[{'choice_id': 0, 'choice': 'The book.', 'choi...","[{'choice_id': 0, 'choice': 'Took.', 'choice_p...","[{'choice_id': 0, 'choice': 'Threw the blanket...","[{'choice_id': 0, 'choice': 'Opened the bag.',...","[{'choice_id': 0, 'choice': 'The sandwich.', '...","[{'choice_id': 0, 'choice': 'The blanket.', 'c...","[{'choice_id': 0, 'choice': 'The book.', 'choi...","[{'choice_id': 0, 'choice': 'Open the door.', ..."
prompt,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...
gen_answer,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...


In [202]:
zs_df['Sequence_T5_3122']['gen_answer']

'Instruction: Predict the answer based on the question.\nQuestion: What did the person do to the pillow after holding the clothes?\nChoices: \n(A) Washed.\n(B) Put down.\n(C) Took.\n(D) Threw.\nAnswer: The answer is  (A).\nExplanation:\nThe person did not put down or take or throw. The person only held. Therefore, the only choice that makes sense is washed.'

In [203]:
zs_df['Sequence_T5_3122']['correct_answer']

'Threw.'

In [204]:
zs_df = zs_df.transpose()

In [205]:
zs_df.reset_index(inplace=True)
zs_df.rename(columns={'index': 'question_id'}, inplace=True)

In [206]:
zs_df

,question_id,video_id,correct_answer,choices,prompt,gen_answer
0,Sequence_T5_3122,APVAD,Threw.,"[{'choice_id': 0, 'choice': 'Washed.', 'choice...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...
1,Sequence_T1_6652,KZODG,The laptop.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...
2,Sequence_T1_2773,2XVR0,The sandwich.,"[{'choice_id': 0, 'choice': 'The phone/camera....",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...
3,Sequence_T3_311,ATV2F,Put down the laptop.,"[{'choice_id': 0, 'choice': 'Took the cup/glas...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...
4,Sequence_T5_3694,AK9IB,Closed.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...
...,...,...,...,...,...,...
196,Sequence_T3_3211,VPBIO,Opened the bag.,"[{'choice_id': 0, 'choice': 'Opened the bag.',...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...
197,Sequence_T2_464,SS3IL,The sandwich.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...
198,Sequence_T2_4514,KRLT7,The clothes.,"[{'choice_id': 0, 'choice': 'The blanket.', 'c...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...
199,Sequence_T1_4306,HNPC8,The closet/cabinet.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...


In [207]:
def extract_answer_from_generated(gen_answer_text):
    match = re.search(r'Answer: The answer is\s+\((.*?)\)\.', gen_answer_text)
    return match.group(1) if match else None

zs_df['parsed_gen_answer'] = zs_df['gen_answer'].apply(extract_answer_from_generated)

In [208]:
zs_df

,question_id,video_id,correct_answer,choices,prompt,gen_answer,parsed_gen_answer
0,Sequence_T5_3122,APVAD,Threw.,"[{'choice_id': 0, 'choice': 'Washed.', 'choice...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A
1,Sequence_T1_6652,KZODG,The laptop.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B
2,Sequence_T1_2773,2XVR0,The sandwich.,"[{'choice_id': 0, 'choice': 'The phone/camera....",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A
3,Sequence_T3_311,ATV2F,Put down the laptop.,"[{'choice_id': 0, 'choice': 'Took the cup/glas...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B
4,Sequence_T5_3694,AK9IB,Closed.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B
...,...,...,...,...,...,...,...
196,Sequence_T3_3211,VPBIO,Opened the bag.,"[{'choice_id': 0, 'choice': 'Opened the bag.',...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A
197,Sequence_T2_464,SS3IL,The sandwich.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C
198,Sequence_T2_4514,KRLT7,The clothes.,"[{'choice_id': 0, 'choice': 'The blanket.', 'c...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C
199,Sequence_T1_4306,HNPC8,The closet/cabinet.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C


In [209]:
def map_correct_answer_to_index(choices, correct_answer):
    for index, choice_dict in enumerate(choices):
        if choice_dict['choice'] == correct_answer:
            return index
    return None  

zs_df['correct_answer_index'] = zs_df.apply(lambda row: map_correct_answer_to_index(row['choices'], row['correct_answer']), axis=1)

In [210]:
zs_df

,question_id,video_id,correct_answer,choices,prompt,gen_answer,parsed_gen_answer,correct_answer_index
0,Sequence_T5_3122,APVAD,Threw.,"[{'choice_id': 0, 'choice': 'Washed.', 'choice...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A,3
1,Sequence_T1_6652,KZODG,The laptop.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B,1
2,Sequence_T1_2773,2XVR0,The sandwich.,"[{'choice_id': 0, 'choice': 'The phone/camera....",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A,3
3,Sequence_T3_311,ATV2F,Put down the laptop.,"[{'choice_id': 0, 'choice': 'Took the cup/glas...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B,3
4,Sequence_T5_3694,AK9IB,Closed.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B,3
...,...,...,...,...,...,...,...,...
196,Sequence_T3_3211,VPBIO,Opened the bag.,"[{'choice_id': 0, 'choice': 'Opened the bag.',...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A,0
197,Sequence_T2_464,SS3IL,The sandwich.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C,0
198,Sequence_T2_4514,KRLT7,The clothes.,"[{'choice_id': 0, 'choice': 'The blanket.', 'c...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C,1
199,Sequence_T1_4306,HNPC8,The closet/cabinet.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C,2


In [211]:
answer_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
zs_df['parsed_gen_answer_index'] = zs_df['parsed_gen_answer'].map(answer_mapping)

In [212]:
zs_df

,question_id,video_id,correct_answer,choices,prompt,gen_answer,parsed_gen_answer,correct_answer_index,parsed_gen_answer_index
0,Sequence_T5_3122,APVAD,Threw.,"[{'choice_id': 0, 'choice': 'Washed.', 'choice...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A,3,0
1,Sequence_T1_6652,KZODG,The laptop.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B,1,1
2,Sequence_T1_2773,2XVR0,The sandwich.,"[{'choice_id': 0, 'choice': 'The phone/camera....",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A,3,0
3,Sequence_T3_311,ATV2F,Put down the laptop.,"[{'choice_id': 0, 'choice': 'Took the cup/glas...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B,3,1
4,Sequence_T5_3694,AK9IB,Closed.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B,3,1
...,...,...,...,...,...,...,...,...,...
196,Sequence_T3_3211,VPBIO,Opened the bag.,"[{'choice_id': 0, 'choice': 'Opened the bag.',...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A,0,0
197,Sequence_T2_464,SS3IL,The sandwich.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C,0,2
198,Sequence_T2_4514,KRLT7,The clothes.,"[{'choice_id': 0, 'choice': 'The blanket.', 'c...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C,1,2
199,Sequence_T1_4306,HNPC8,The closet/cabinet.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C,2,2


In [213]:
zs_df['is_correct'] = zs_df['parsed_gen_answer_index'] == zs_df['correct_answer_index']


In [214]:
print(zs_df[['question_id', 'parsed_gen_answer', 'correct_answer', 'parsed_gen_answer_index', 'correct_answer_index', 'is_correct']])


           question_id parsed_gen_answer        correct_answer  \
0     Sequence_T5_3122                 A                Threw.   
1     Sequence_T1_6652                 B           The laptop.   
2     Sequence_T1_2773                 A         The sandwich.   
3      Sequence_T3_311                 B  Put down the laptop.   
4     Sequence_T5_3694                 B               Closed.   
..                 ...               ...                   ...   
196   Sequence_T3_3211                 A       Opened the bag.   
197    Sequence_T2_464                 C         The sandwich.   
198   Sequence_T2_4514                 C          The clothes.   
199   Sequence_T1_4306                 C   The closet/cabinet.   
200  Prediction_T1_512                 A     Sit at the table.   

     parsed_gen_answer_index  correct_answer_index  is_correct  
0                          0                     3       False  
1                          1                     1        True  
2           

In [215]:
zs_df['question_type'] = zs_df['question_id'].apply(lambda x: x.split('_')[0])

In [216]:
zs_df

,question_id,video_id,correct_answer,choices,prompt,gen_answer,parsed_gen_answer,correct_answer_index,parsed_gen_answer_index,is_correct,question_type
0,Sequence_T5_3122,APVAD,Threw.,"[{'choice_id': 0, 'choice': 'Washed.', 'choice...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A,3,0,False,Sequence
1,Sequence_T1_6652,KZODG,The laptop.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B,1,1,True,Sequence
2,Sequence_T1_2773,2XVR0,The sandwich.,"[{'choice_id': 0, 'choice': 'The phone/camera....",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A,3,0,False,Sequence
3,Sequence_T3_311,ATV2F,Put down the laptop.,"[{'choice_id': 0, 'choice': 'Took the cup/glas...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B,3,1,False,Sequence
4,Sequence_T5_3694,AK9IB,Closed.,"[{'choice_id': 0, 'choice': 'Threw.', 'choice_...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,B,3,1,False,Sequence
...,...,...,...,...,...,...,...,...,...,...,...
196,Sequence_T3_3211,VPBIO,Opened the bag.,"[{'choice_id': 0, 'choice': 'Opened the bag.',...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,A,0,0,True,Sequence
197,Sequence_T2_464,SS3IL,The sandwich.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C,0,2,False,Sequence
198,Sequence_T2_4514,KRLT7,The clothes.,"[{'choice_id': 0, 'choice': 'The blanket.', 'c...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C,1,2,False,Sequence
199,Sequence_T1_4306,HNPC8,The closet/cabinet.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",Instruction: Predict the answer based on the q...,Instruction: Predict the answer based on the q...,C,2,2,True,Sequence


In [217]:
correctness_counts_by_type = zs_df.groupby('question_type')['is_correct'].value_counts().unstack(fill_value=0)
correctness_counts_by_type

is_correct,False,True
question_type,,
Feasibility,12,3
Prediction,19,7
Sequence,125,35


In [218]:
counts_by_type = pd.crosstab(zs_df['question_type'], zs_df['is_correct'])


In [219]:
counts_by_type = counts_by_type.reindex(columns=[False, True], fill_value=0)
counts_by_type

is_correct,False,True
question_type,,
Feasibility,12,3
Prediction,19,7
Sequence,125,35


In [220]:
percentage_correct = counts_by_type[True] / counts_by_type.sum(axis=1) * 100
percentage_correct

question_type
Feasibility    20.000000
Prediction     26.923077
Sequence       21.875000
dtype: float64

In [230]:
zeroShotProgressPath = "ZeroShotOutput/ZeroShotOutputIndex550.json"
with open(zeroShotProgressPath, "r") as file:
    ZS_data = json.load(file)
zs_df = pd.DataFrame(ZS_data)
zs_df = zs_df.transpose()
zs_df.reset_index(inplace=True)
zs_df.rename(columns={'index': 'question_id'}, inplace=True)

zs_df['parsed_gen_answer'] = zs_df['gen_answer'].apply(extract_answer_from_generated)

zs_df['correct_answer_index'] = zs_df.apply(lambda row: map_correct_answer_to_index(row['choices'], row['correct_answer']), axis=1)
answer_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
zs_df['parsed_gen_answer_index'] = zs_df['parsed_gen_answer'].map(answer_mapping)
zs_df['is_correct'] = zs_df['parsed_gen_answer_index'] == zs_df['correct_answer_index']
zs_df['question_type'] = zs_df['question_id'].apply(lambda x: x.split('_')[0])
correctness_counts_by_type = zs_df.groupby('question_type')['is_correct'].value_counts().unstack(fill_value=0)
counts_by_type = pd.crosstab(zs_df['question_type'], zs_df['is_correct'])
counts_by_type = counts_by_type.reindex(columns=[False, True], fill_value=0)
print(counts_by_type)
percentage_correct = counts_by_type[True] / counts_by_type.sum(axis=1) * 100
print(percentage_correct)

counts_by_type['Percentage Correct'] = (counts_by_type[True] / counts_by_type.sum(axis=1) * 100).round(1).astype(str) + '%'
print(counts_by_type)
print('\n\n\n\n')
counts_by_type.columns = ['Number Incorrect', 'Number Correct', 'Percentage Correct']
counts_by_type.loc['Interaction'] = [4384, 1405, '32.0%']

total_row = counts_by_type.sum()
total_row['Percentage Correct'] = f"{(total_row['Number Correct'] / (total_row['Number Correct'] + total_row['Number Incorrect']) * 100).round(1)}%"
counts_by_type.loc['Total'] = total_row
latex_output = counts_by_type.to_latex(
    escape=False,  # Prevents converting special characters like underscores to LaTeX commands
    column_format='|l|c|c|c|',  # Table column alignment
    index_names=True,
    header=True
)
print(latex_output)


is_correct     False  True 
question_type              
Feasibility       36      6
Prediction        50     26
Sequence         317    116
question_type
Feasibility    14.285714
Prediction     34.210526
Sequence       26.789838
dtype: float64
is_correct     False  True Percentage Correct
question_type                                
Feasibility       36     6              14.3%
Prediction        50    26              34.2%
Sequence         317   116              26.8%





\begin{tabular}{|l|c|c|c|}
\toprule
{} &  Number Incorrect &  Number Correct & Percentage Correct \\
question_type &                   &                 &                    \\
\midrule
Feasibility   &                36 &               6 &              14.3% \\
Prediction    &                50 &              26 &              34.2% \\
Sequence      &               317 &             116 &              26.8% \\
Interaction   &              4384 &            1405 &              32.0% \\
Total         &              47

/tmp/ipykernel_546068/3718888826.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_output = counts_by_type.to_latex(


In [227]:
counts_by_type

,Number Incorrect,Number Correct,Percentage Correct
question_type,,,
Feasibility,36,6,14.3%
Prediction,50,26,34.2%
Sequence,317,116,26.8%
Interaction,4384,1405,32.0%
Total,4787,1553,24.5%
